In [1]:
from pygraylog.graylogapi import GraylogAPI
import json
import sys
import numpy as np
import pandas as pd
import datetime

In [2]:
log_level = 0

graylog_con={
        "user":"pavel",
        "password":"O777ver7k1",
        "url":"http://graylog-ui.roamability.com:9000/api",
        "offset":1,
        "limit":300
    }

api = GraylogAPI(
    graylog_con["url"],
    graylog_con["user"], 
    graylog_con["password"])

In [86]:
#--------------------------------------------------------------------------------
#Form date and time
#--------------------------------------------------------------------------------
#format like 2017-03-06T16:28

#Manually set the date and time
dateFrom = datetime.datetime.strptime('2018-01-12T05:00', '%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime('2018-01-12T12:00', '%Y-%m-%dT%H:%M')

#The last period of time
dateFrom = datetime.datetime.strptime((datetime.datetime.now()-datetime.timedelta(hours=12)).strftime("%Y-%m-%dT%H:%M"),'%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M"), '%Y-%m-%dT%H:%M')

#--------------------------------------------------------------------------------
#Form query
#--------------------------------------------------------------------------------
#Fields examples
#protocol:ss7
#tcap_otid:"a2:2a:0c:40"
#tcap_dtid:"a2:2a:0c:40"
#opCode_localValue:2
#TCAP_type:Begin
#e212_imsi:425019629870706
#tcap_result:1
#tcap_dialogue_service_user:2

#TID
#query='protocol:ss7 AND (tcap_otid:"a2:2a:0c:40" OR tcap_dtid:"a2:2a:0c:40")'

#Opcode, TCAP_type and IMSI
#query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425019629870706'
#query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin'

#Wrong MTFSM MAP version
#query='protocol:ss7 AND tcap_result:1 AND tcap_dialogue_service_user:2 AND tcap_application_context_name:"0.4.0.0.1.0.25.2" AND (m3ua_OPC:1961 OR m3ua_OPC:1962)'

query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:"a2:2a:0c:40"'

#--------------------------------------------------------------------------------
#Run the query
#--------------------------------------------------------------------------------
gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo, offset=graylog_con["offset"], limit=graylog_con["limit"])


'protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:"a2:2a:0c:40"'

In [4]:
def outLog(message, level=1):
    if(level <= log_level):
        print (message)
        sys.stdout.flush()

def LoadGraylogResult(exportsearch_results):
        jsonResult = json.loads(exportsearch_results)
        outputResult = list()
        i=0
        if "total_results" in jsonResult:
            outLog("Graylog search found {} messages".format(jsonResult["total_results"]),0)

        # Get the results and store them
            for result in jsonResult["messages"]:  
                if "message" in  result and isinstance(result["message"], dict):
                    outputResult.append(result["message"])
                    i+=1
                    if i%100==0: outLog("loaded {} records from Graylog".format(i),0)
            outLog("Graylog search load {} messages".format(len(outputResult)),0)
        else:
            outLog("Graylog search error: {}".format(exportsearch_results),0)
        return outputResult
    
data=LoadGraylogResult(gres)

Graylog search found 0 messages
Graylog search load 0 messages


In [109]:
#Загружаем данные в df

columns=list(data[0].keys())
values = list(data[0].values())
arr_len = len(values)
df=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
for i in range(len(data)-1):
    columns=list(data[i+1].keys())
    values = list(data[i+1].values())
    arr_len = len(values)
    df1=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
    df=pd.concat([df, df1],axis=0,join='outer')
df=df.reset_index(drop=True)

if 'tcap_otid' in df.columns:
    df['tcap_otid']=df['tcap_otid'].fillna(0)
if 'tcap_dtid' in df.columns:
    df['tcap_dtid']=df['tcap_dtid'].fillna(0)
    
fields=[
    'm3ua_OPC','m3ua_DPC',
    'sccp_calling_nai','sccp_calling_digits','sccp_calling_ri','sccp_calling_ssn',
    'sccp_called_nai','sccp_called_digits','sccp_called_ri','sccp_called_ssn',
    'sccp_class','sccp_message_type','protocol_name','protocol','protocol_version','opCode_localValue','TCAP_type',
    'tcap_otid','tcap_dtid',
    'e164_msisdn','e212_imsi']

xdr_fields=[]
for a in fields:
    if a in df.columns:
        #xdr_fields+=a
        xdr_fields.append(a)

df[xdr_fields].head(3)

,m3ua_OPC,m3ua_DPC,sccp_calling_nai,sccp_calling_digits,sccp_calling_ri,sccp_calling_ssn,sccp_called_nai,sccp_called_digits,sccp_called_ri,sccp_called_ssn,...,sccp_message_type,protocol_name,protocol,protocol_version,opCode_localValue,TCAP_type,tcap_otid,tcap_dtid,e164_msisdn,e212_imsi
0,1962,2505,4,233200005214,0,146,4,97254120624,0,146,...,0x00000009,camel,ss7,1,NaN,Continue,03:ec:66:44,a1:1a:0e:c7,NaN,NaN
1,2505,1961,4,97254120624,0,146,4,233200005214,0,146,...,0x00000009,camel,ss7,2,NaN,Continue,a1:1a:0e:c7,03:ec:66:44,233244987328,NaN
2,7138,700,4,972559900050,0,6,4,972559900040,0,149,...,0x00000009,map,ss7,3,56,End,0,a1:1a:0e:c7,NaN,NaN


In [110]:
#Функция для сборки транзакции

def xdr_compile(otid,ind,df_xdr,df):
    
    fields=['tcap_dtid','tcap_otid','TCAP_type']
    if all(item in df.columns for item in fields):
        dtid=111
        df_xdr_temp=df_xdr
        for i in range(ind-1,-1,-1):
            if df.iloc[i]['tcap_dtid']==otid and df.iloc[i]['tcap_otid']==0\
            and (df.iloc[i]['TCAP_type']=='End' or df.iloc[i]['TCAP_type']=='Abort'):
                values = list(df.iloc[i].values)
                arr_len = len(values)
                df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
                df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
                return df_xdr_temp
            elif df.iloc[i]['tcap_dtid']==otid and df.iloc[i]['TCAP_type']=='Continue' and df.iloc[i]['tcap_otid']<>dtid:
                dtid=df.iloc[i]['tcap_otid']
                values = list(df.iloc[i].values)
                arr_len = len(values)
                df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
                df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
            elif df.iloc[i]['TCAP_type']=='Continue'\
            and (df.iloc[i]['tcap_dtid']==otid or df.iloc[i]['tcap_otid']==otid or df.iloc[i]['tcap_dtid']==dtid or df.iloc[i]['tcap_otid']==dtid):
                values = list(df.iloc[i].values)
                arr_len = len(values)
                df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
                df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
            elif (df.iloc[i]['TCAP_type']=='End' or df.iloc[i]['TCAP_type']=='Abort')\
            and (df.iloc[i]['tcap_dtid']==otid or df.iloc[i]['tcap_otid']==otid or df.iloc[i]['tcap_dtid']==dtid or df.iloc[i]['tcap_otid']==dtid):
                values = list(df.iloc[i].values)
                arr_len = len(values)
                df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
                df_xdr_temp=pd.concat([df_xdr_temp, df_temp],axis=0,join='outer')
                return df_xdr_temp
        return df_xdr_temp
    else:
        print 'Not all fields are in the DataFrame'

In [111]:
#Сборка датафрейма XDR

fields=['tcap_dtid','tcap_otid','TCAP_type']

if all(item in df.columns for item in fields):
    columns=list(df.keys())
    df_xdr_res=pd.DataFrame(columns=columns)
    for i in range(len(df)-1,-1,-1):
        #if df.iloc[i]['TCAP_type']=='Begin' and (df.iloc[i]['e212_imsi']=='425019612512015'):
        if df.iloc[i]['TCAP_type']=='Begin':
            values = list(df.iloc[i].values)
            arr_len = len(values)
            df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
            df_xdr_res=pd.concat([df_xdr_res,xdr_compile(df.iloc[i]['tcap_otid'],i,df_temp,df)],axis=0,join='outer')
else:
    print 'Not all fields are in the DataFrame'

df_xdr_res=df_xdr_res.reset_index(drop=True)

df_xdr_res[xdr_fields]

,m3ua_OPC,m3ua_DPC,sccp_calling_nai,sccp_calling_digits,sccp_calling_ri,sccp_calling_ssn,sccp_called_nai,sccp_called_digits,sccp_called_ri,sccp_called_ssn,...,sccp_message_type,protocol_name,protocol,protocol_version,opCode_localValue,TCAP_type,tcap_otid,tcap_dtid,e164_msisdn,e212_imsi
0,700,7138,4,972559900040,0,7,4,972559912012424,0,6,...,0x00000009,map,ss7,3,56,Begin,a1:1a:0e:c7,0,NaN,425191201242462
1,7138,700,4,972559900050,0,6,4,972559900040,0,7,...,0x00000009,map,ss7,3,56,End,0,a1:1a:0e:c7,NaN,NaN
2,800,781,4,233200005213,0,7,4,972549612531490,0,6,...,0x00000009,map,ss7,NaN,56,Begin,a1:1a:0e:c7,0,NaN,425019612531490
3,781,800,4,97254120634,0,6,4,233200005213,0,7,...,0x00000009,map,ss7,NaN,56,End,0,a1:1a:0e:c7,NaN,NaN
4,700,7138,4,972559900040,0,8,4,972559813409,0,6,...,0x00000009,map,ss7,NaN,45,Begin,a1:1a:0e:c7,0,972559900040,NaN
5,7138,700,4,972559900050,0,6,4,972559900040,0,8,...,0x00000009,map,ss7,NaN,27,End,0,a1:1a:0e:c7,NaN,NaN
6,1093,4408,4,85263347736,0,149,4,972549611011686,0,6,...,0x00000009,map,ss7,NaN,56,Begin,a1:1a:0e:c7,0,NaN,425019611011686
7,1961,2506,4,550541102000,0,6,4,550549900000,0,7,...,0x00000009,map,ss7,3,56,End,0,a1:1a:0e:c7,NaN,NaN
8,2505,1961,4,550549900000,0,7,4,550540540398491,0,6,...,0x00000009,map,ss7,3,56,Begin,a1:1a:0e:c7,0,NaN,724540540398491
9,1961,2506,4,550541102000,0,6,4,550549900000,0,7,...,0x00000009,map,ss7,3,56,End,0,a1:1a:0e:c7,NaN,NaN


In [106]:
#The last period of time
dateFrom = datetime.datetime.strptime((datetime.datetime.now()-datetime.timedelta(hours=4)).strftime("%Y-%m-%dT%H:%M"),'%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M"), '%Y-%m-%dT%H:%M')

#Opcode, TCAP_type and IMSI
query='protocol:ss7 AND opCode_localValue:2* AND TCAP_type:Begin AND e212_imsi:425019612505402'

#--------------------------------------------------------------------------------
#Run the query
#--------------------------------------------------------------------------------
gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo, offset=graylog_con["offset"], limit=graylog_con["limit"])

data=LoadGraylogResult(gres)

Graylog search found 4 messages
Graylog search load 3 messages


In [107]:
tid_list=[]
str_temp=''

for i in range(len(data)):
    tid_list.append(data[i]['tcap_otid'])
tid_list
tid_list=tuple(tid_list)

i=1
for a in tid_list:
    str_temp=str_temp + 'tcap_otid:' + '"' + a  + '"' + ' OR tcap_dtid:' + '"' + a  + '"'
    if i<len(tid_list):
        str_temp=str_temp + ' OR '
    i=i+1
query=str('protocol:ss7 AND (' + str_temp + ')')
query

'protocol:ss7 AND (tcap_otid:"54:95:00:0a" OR tcap_dtid:"54:95:00:0a" OR tcap_otid:"a1:1a:0e:c7" OR tcap_dtid:"a1:1a:0e:c7" OR tcap_otid:"b2:00:01:f3" OR tcap_dtid:"b2:00:01:f3")'

In [108]:
gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo, offset=graylog_con["offset"], limit=graylog_con["limit"])
data=LoadGraylogResult(gres)

Graylog search found 32 messages
Graylog search load 31 messages
